In [1]:
from load_model_and_data import loader, known_skew
from bootstrapper import data_iterator, data_iterator_2d
import numpy as np
from os.path import join
import itertools
import pandas as pd
import collections

In [2]:
wofs_domain = pd.read_pickle('wofs_domains.pkl').astype(int)
cent_lon = wofs_domain['cent_lon']
cent_lat = wofs_domain['cent_lat']
all_wofs_dates = wofs_domain['dates']

In [3]:
east_west_divider = np.where(np.round(cent_lon,5) <= -90.0, 'west', 'east')
north_south_divider = np.where(np.round(cent_lat,5) >= 37.0, 'north', 'south')
divider = np.array([f'{a}_{b}' for a,b in zip(north_south_divider, east_west_divider)], dtype='object')

In [4]:
iterator = ['Northern Great Plains', 'Mid-Atlantic', 'Southern Great Plains', 'South East']
mapper = {'north_west' : 'Northern Great Plains', 
          'north_east' : 'Mid-Atlantic', 
          'south_west' : 'Southern Great Plains', 
          'south_east' : 'South East'
         }

In [5]:
lead_times = ['first_hour', 'second_hour']
targets = ['tornado', 'severe_hail', 'severe_wind']
n_bootstrap=500

for lead_time, target in itertools.product(lead_times, targets):
    (predictions, 
     X, y, 
     init_times, 
     test_dates,
    fti) = loader(lead_time, target)

    geography = np.zeros((len(X)), dtype='object')
    for a,b in itertools.product(['north', 'south'], ['east', 'west']):
        cardinality = f'{a}_{b}'
        date_subset = list(all_wofs_dates[divider==cardinality])
        geography[test_dates.astype(int).isin((date_subset))] = cardinality
    
    geography = np.array([mapper[item] for item in geography], dtype='object')
    
    X['Run Date'] = test_dates
    # Stratify by initialization times 
    datasets = data_iterator(to_be_stratified=geography, 
                             iterator=iterator,
                             predictions=predictions,
                             X=X,
                             y=y,
                             known_skew=known_skew[lead_time][target],
                             n_bootstrap=n_bootstrap,
                            )

    for ds in datasets:
        ds['xdata'] = iterator
    
    # Save the results
    out_path = '/work/mflora/ML_DATA/stratified_results'
    for ds, mode in zip(datasets, ['ml', 'ml_reduced', 'bl']):
        ds.to_netcdf(join(out_path, f'{mode}_geographic_stratify_{lead_time}_{target}.nc'))
        ds.close()  

/home/monte.flora/miniconda3/envs/wofs_test/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator SimpleImputer from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/monte.flora/miniconda3/envs/wofs_test/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/monte.flora/miniconda3/envs/wofs_test/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from vers

/home/monte.flora/miniconda3/envs/wofs_test/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator SimpleImputer from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/monte.flora/miniconda3/envs/wofs_test/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/monte.flora/miniconda3/envs/wofs_test/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from vers